In [2]:
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

## Tree Node Class

In [3]:
class treeNode:
    def __init__(self, nameVal, numOccurence, parentNode):
        self.name = nameVal
        self.count = numOccurence
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    
    def inc(self, numOccurence):
        self.count += numOccurence
    
    def display(self, ind=1):
        print(' '*ind + str(self.name)+' ' + str(self.count))
        for child in self.children.values():
            child.display(ind+1)

In [4]:
root = treeNode('pyramid', 9, None)
root.children['eye'] = treeNode('eye', 13, None)
root.children['php'] = treeNode('php', 11, None)
root.display()

 pyramid 9
  php 11
  eye 13


## Create Tree

In [21]:
def createTree(dataDic, minSup = 1):
    # count the frequency for each item
    headerTable = {}
    for trans, count in dataDic.items():
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + count
    
    # filter the items whos frequency less than minSup
    for k in headerTable.keys():
        if headerTable[k] < minSup:
            del headerTable[k]
            
    # get the frequent item set
    freqItemsSet = set(headerTable.keys())
    if(len(freqItemsSet) == 0):
        return None, None
    
    # reshape the headerTable
    for k in headerTable.keys():
        headerTable[k] = [headerTable[k], None] # [the frequency of item, the first node in link list]
    #print(headerTable)
    # create tree
    retTree = treeNode('Null Set', 1, None)
    for trans, count in dataDic.items():
        # sort the items in each transaction by the frequency
        localDic = {} # localDic[item] = item frequency
        for item in trans:
            if item in freqItemsSet:
                localDic[item] = headerTable[item][0]
        if(len(localDic) > 0):
            orderedItems = [ v[0] for v in sorted(localDic.items(), key = lambda p : p[1], reverse = True) ]
            #print(orderedItems)
            updateTree(orderedItems, retTree, headerTable, count) # insert new transaction into FP tree
    
    return retTree, headerTable
        
    
def updateTree(items, retTree, headerTable, count):
    # the first item 
    if items[0] in retTree.children:
        retTree.children[items[0]].inc(count)
    else:
        retTree.children[items[0]] = treeNode(items[0], count, retTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = retTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], retTree.children[items[0]])
    
    # if there are any other node except the first item, insert into FP tree recursion
    if len(items) > 1:
        updateTree(items[1::], retTree.children[items[0]], headerTable, count)
    
def updateHeader(nodeToSim, targetNode):
    while nodeToSim.nodeLink != None:
        nodeToSim = nodeToSim.nodeLink
    nodeToSim.nodeLink = targetNode

In [6]:
def loadSimpDataSet():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataList):
    retDit = {}
    for trans in dataList:
        froz = frozenset(trans)
        retDit[froz] = retDit.get(froz, 0) + 1
    return retDit

In [7]:
dataList = loadSimpDataSet()
dataDic = createInitSet(dataList)
dataDic

{frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1}

In [8]:
fpTree, headTabel = createTree(dataDic, 3)
fpTree.display()

['z', 'x', 'y', 's', 't']
['x', 's', 'r']
['z', 'x', 'y', 's', 't']
['z', 'x', 'y', 'r', 't']
['z', 'r']
['z']
 Null Set 1
  x 1
   s 1
    r 1
  z 5
   x 3
    y 3
     s 2
      t 2
     r 1
      t 1
   r 1


## Mining frequent tree

In [16]:
def ascendTree(leafNode, prefixPath):
    if( leafNode.parent != None ):
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

def findPrefixPath(basePath, treeNode):
    conditionalPaths = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        #print(prefixPath)
        if len(prefixPath) > 1:
            conditionalPaths[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return conditionalPaths

def mineTree(inTree, headerTable, minSup, prefixItems, freqItemList):
    bigL = [ v[0] for v in sorted(headerTable.items(), key=lambda p : p[1]) ]
    for basePath in bigL:
        newFreqSet = prefixItems.copy()
        newFreqSet.add(basePath)
        freqItemList.append(newFreqSet)
        conditionalPaths = findPrefixPath(basePath, headerTable[basePath][1])
        conditionalTree, conditionalHeadTable = createTree(conditionalPaths, minSup)
        
        if conditionalHeadTable != None:
            mineTree(conditionalTree, conditionalHeadTable, minSup, newFreqSet, freqItemList)

In [18]:
print(findPrefixPath('x', headTabel['x'][1]))
print(findPrefixPath('z', headTabel['z'][1]))
print(findPrefixPath('r', headTabel['r'][1]))

{frozenset(['z']): 3}
{}
{frozenset(['x', 's']): 1, frozenset(['z']): 1, frozenset(['y', 'x', 'z']): 1}


In [22]:
fpTree, headTabel = createTree(dataDic, 3)
fpTree.display()
freqItemList = []
mineTree(fpTree, headTabel, 3, set([]), freqItemList)
print('\n')
print(freqItemList)

 Null Set 1
  x 1
   s 1
    r 1
  z 5
   x 3
    y 3
     s 2
      t 2
     r 1
      t 1
   r 1


[set(['t']), set(['y', 't']), set(['z', 't']), set(['x', 'z', 't']), set(['y', 'x', 'z', 't']), set(['y', 'z', 't']), set(['x', 't']), set(['y', 'x', 't']), set(['y']), set(['y', 'z']), set(['y', 'x', 'z']), set(['y', 'x']), set(['r']), set(['s']), set(['x', 's']), set(['x']), set(['x', 'z']), set(['z'])]


## A real case:

In [27]:
DATA_DIR = "data/fpGrowth/"
dataList = [line.split() for line in open(DATA_DIR + 'kosarak.dat').readlines()]
dataDic = createInitSet(dataList)
dataDic.items()[:5]

[(frozenset({'1',
             '1057',
             '11',
             '136',
             '1601',
             '259',
             '3145',
             '368',
             '3957',
             '4',
             '4598',
             '56',
             '6',
             '764',
             '77'}),
  1),
 (frozenset({'11', '40', '4674', '6'}), 1),
 (frozenset({'11', '1450', '2'}), 1),
 (frozenset({'1130',
             '148',
             '15030',
             '1607',
             '218',
             '3297',
             '5143',
             '5688',
             '6',
             '7',
             '772',
             '838',
             '885'}),
  1),
 (frozenset({'1', '2131', '3'}), 1)]

In [28]:
fpTree, headerTable = createTree(dataDic, 100000)

In [29]:
freqItemList = []
mineTree(fpTree, headerTable, 100000, set([]), freqItemList)

In [30]:
freqItemList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '3'},
 {'11', '3', '6'},
 {'11', '6'},
 {'3'},
 {'3', '6'},
 {'6'}]